# Making a List of all the paratransit Agencies in the state 

per bill requirements, we need to make a list of all paratransit agencies in the state and include everyone who gets mills-alquish-deddeh act funding. 

## Sources
* Trillium / Cal ITP List of Transit Agencies 
* Blackcat Dump 
* [State Controller Website](https://www.sco.ca.gov/ard_payments_transit.html)

In [9]:
import pandas as pd 
import json
import sqlalchemy

## Parsing Blackcat

In [10]:
with open('./data/blackcat_dump_may_1.json','r') as d:
    data = json.load(d)

the data structure is a list of tables, each with a JSON object w/ tbl name and rows as a key

Let's parse this into a series of pandas dataframes in a diction w/ the table name as keys

In [11]:
dfs_dict = {}
for table in data['Tables']:
    tbl_name = table['Name']
    try:
        col_list = [k['Column'] for k in table['Rows'][0]['Columns']]
    except IndexError:
        print(f"There are no columns in {tbl_name}")
    rows = []
    for row in table['Rows']:
        values = [d['Value'] for d in row['Columns']]
        rows.append(values)
    dfs_dict[tbl_name] = pd.DataFrame(rows, columns=col_list)


There are no columns in BudgetExpense
There are no columns in BudgetExpenseComments
There are no columns in BudgetExpenseReports
There are no columns in data_Admendment_View
There are no columns in data_BudgetItemsTotals
There are no columns in data_BudgetItemsTotals_Report
There are no columns in data_BudgetItems_Report
There are no columns in data_Budget_Reporting_ServiceData
There are no columns in data_Certification
There are no columns in data_Comment
There are no columns in data_ContractPaymentRequest
There are no columns in data_ContractPaymentRequest_Allocation
There are no columns in data_ContractPaymentRequest_AllocationDistrobution
There are no columns in data_ContractPaymentRequest_CodeStrip
There are no columns in data_ContractPaymentRequest_CodeStripInvoice
There are no columns in data_ContractPaymentRequest_Documents
There are no columns in data_ContractReport5307
There are no columns in data_ContractReports
There are no columns in data_ContractReportsOTPPP
There are no 

In [12]:
print(f"The total number of parsed tables is {len(dfs_dict)}")

The total number of parsed tables is 375


In [13]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///./data/blackcat.sqlite')

In [15]:
{v.to_sql(k, con=engine) for k,v in dfs_dict.items()}

{None}

## Determining what Agencies are paratransit from blackcat

In [22]:
for k,v in dfs_dict.items():
    df = v
    if len(df) == 0:
        display(f"no data in {k}")
    else: 
        display(k)
        display(df.columns)
        display(df.head())


'ALI'

Index(['ALI', 'Code', 'Description', 'id'], dtype='object')

,ALI,Code,Description,id
0,ALI,111102,ENG & DESIGN STD 35-FT BUS,1
1,ALI,111103,ENG & DESIGN STD 30-FT BUS,2
2,ALI,111104,ENG & DESIGN <30-FT BUS,3
3,ALI,111105,ENG & DESIGN SCHOOL BUS,4
4,ALI,111106,ENG & DESIGN ARTICULATED BUS,5


'ALIGroup'

Index(['ALI Group', 'Code', 'Description'], dtype='object')

...

Index(['VehicleId', 'OrganizationId', 'LicensePlate', 'VIN', 'VehicleIDNumber',
       'FundedByDOT', 'FundingSource', 'FundingAgency', 'GrantNumber',
       'StandardAgreementNumber', 'FederalShare', 'FederalSharePercent',
       'TollCredit', 'TollCreditPercent', 'LocalShare', 'LocalSharePercent',
       'TotalCost', 'FTACategory', 'Year', 'Manufacturer', 'Model', 'Chassis',
       'Class', 'FuelType', 'VehicleLength', 'AmbulatorySeatingCapacity',
       'WheelchairPositions', 'WheelchairEquipped', 'VehicleStatus',
       'ImprovementType', 'CurrentCondition', 'CurrentMileage', 'VehicleUse',
       'PurchaseOrderNumber', 'PurchaseOrderDate', 'InvoiceNumber', 'Vendor',
       'ReceivedDate', 'LastInspectionDate', 'NextInspectionDate',
       'StateLienHolder', 'TitleNumber', 'VehicleTitleReleaseDate',
       'VehicleReleaseReportingDate', 'FairMarketValue', 'TransferDate',
       'SoldPrice', 'TransferRecipent', 'TransferStandardAgreementNumber',
       'ReplacementVIN', 'InServiceDat

,VehicleId,OrganizationId,LicensePlate,VIN,VehicleIDNumber,FundedByDOT,FundingSource,FundingAgency,GrantNumber,StandardAgreementNumber,...,OutServiceDate,NTDID,Comments,Sys_ModifiedUserID,NTD_VehicleType,NTD_DedicatedFleet,NTD_OwnershipType,NTD_FundingType,NTD_Mode,NTD_TOS
0,,3661,,15GGB271X91177352,,Yes,Section 5311,Federal,CA-18-X025,647001,...,,,,481,,,,,,
1,,3661,,15GGB271191177353,,Yes,Section 5311,Federal,CA-18-X025,647001,...,,,,481,,,,,,
2,4591,3662,1352672,15GGB2710E1183297,,Yes,Section 5311,Federal,CA-18-X059,643001,...,,,,481,,,,,,
3,,3662,,15GGB271691176019,,Yes,Section 5311,Federal,CA-18-X028,648006,...,,,,481,,,,,,
4,,3662,,15GGB191171077693,,Yes,Section 5311,Federal,CA-18-X023,645007,...,,,,481,,,,,,


'_Agency_Contact'

Index(['District', 'Agency', 'RTPA/MPO', 'Title', 'Contact Person',
       'First Name', 'Mid Initial', 'Last Name', 'Role', 'Email', 'Phone',
       'Alt Contact Information', 'F13', 'F14', 'F15'],
      dtype='object')

,District,Agency,RTPA/MPO,Title,Contact Person,First Name,Mid Initial,Last Name,Role,Email,Phone,Alt Contact Information,F13,F14,F15
0,2,Agency on Aging for Lake and Mendocino Counties,,Ms.,Carol Hutchinson,Carol,,Hutchinson,,,707-643-7894,,,,
1,2,Agency on Aging for Lake and Mendocino Counties,,Mr.,Mike Parkinson,Mike,,Parkinson,,parkinsonm@mendocino.ca.us,707-643-7894,,,,
2,,Alpha Project for the Homeless,,Ms.,Amy Gonyeau,Amy,,Gonyeau,,amy@alphaproject.org,619-542-1877,,,,
3,,Alpha Project for the Homeless,,Mr.,Robert McElroy,Robert,,McElroy,,robert@alphaproject.org,619-542-1877,,,,
4,,Amador Regional Transit System,,Ms.,Carla Meyer,Carla,,Meyer,,Carla@amadortransit.com,209-267-5089,,,,


which tables are useful

In [49]:
num_rows_df = pd.DataFrame.from_dict({k: len(df) for k, df in dfs_dict.items()}, orient='index').reset_index().rename({'index': 'table_names', 0: 'num_rows'}, axis=1)

In [54]:
num_rows_df.sort_values(by='num_rows',ascending=False).head(20)

,table_names,num_rows
49,data_DynamicFields_Template,36738
90,data_Notification_QueuedMessages,29851
72,data_GrantOpportunityForms,21994
91,data_Notification_QueueGroups,16451
47,data_Documents,14266
354,ProjectTaskBudgetLines,9974
355,ProjectTaskBudgetLinesExtended,8956
131,data_UnionAffiliations,6675
54,data_Entity_PropertiesRecords,6413
174,Fact_ApplicationDocument_Log,5157
